In [1]:
import csv
import mysql.connector
from mysql.connector import Error
from datetime import datetime

In [ ]:


# Conexión a MySQL
config = {
    'user': 'root', #nombre de usuario
    'password': 'admin',#contraseña
    'host': 'localhost',
    'database': 'coches',# nombre de la base de datos
    'raise_on_warnings': True
}

def create_connection():
    """Crear conexión a la base de datos"""
    try:
        conn = mysql.connector.connect(**config)
        return conn
    except Error as e:
        print(f"Error al conectar a MySQL: {e}")
        return None


def insert_ubicacion(conn, ciudad):
    """Insertar ubicación si no existe y devolver ID"""
    cursor = conn.cursor()
    try:
        # Verificar si ya existe
        cursor.execute("SELECT id FROM ubicaciones WHERE ciudad = %s", (ciudad,))
        result = cursor.fetchone()
        
        if result:
            return result[0]
        else:
            # Insertar nueva ubicación
            cursor.execute("INSERT INTO ubicaciones (ciudad) VALUES (%s)", (ciudad,))
            conn.commit()
            return cursor.lastrowid
    except Error as e:
        print(f"Error al insertar ubicación: {e}")
        return None
    finally:
        cursor.close()

def insert_tipo_carroceria(conn, nombre):
    """Insertar tipo de carrocería si no existe y devolver ID"""
    if not nombre or nombre == 'Desconocido':
        return None
        
    cursor = conn.cursor()
    try:
        # Verificar si ya existe
        cursor.execute("SELECT id FROM tipos_carroceria WHERE nombre = %s", (nombre,))
        result = cursor.fetchone()
        
        if result:
            return result[0]
        else:
            # Insertar nuevo tipo
            cursor.execute("INSERT INTO tipos_carroceria (nombre) VALUES (%s)", (nombre,))
            conn.commit()
            return cursor.lastrowid
    except Error as e:
        print(f"Error al insertar tipo de carrocería: {e}")
        return None
    finally:
        cursor.close()

def to_boolean(value):
    """Convertir valor a booleano"""
    if isinstance(value, bool):
        return value
    if isinstance(value, (int, float)):
        return bool(value)
    if isinstance(value, str):
        return value.lower() in ('true', '1', 't', 'y', 'yes')
    return False

def to_float(value):
    """Convertir valor a float seguro"""
    try:
        return float(value) if value else None
    except (ValueError, TypeError):
        return None

def to_int(value):
    """Convertir valor a int seguro"""
    try:
        return int(float(value)) if value else None
    except (ValueError, TypeError):
        return None

def process_csv_file(csv_file):
    """Procesar el archivo CSV y cargar los datos en la base de datos"""
    conn = create_connection()
    if not conn:
        return
    
    
    cursor = conn.cursor(dictionary=True)
    
    with open(csv_file, mode='r', encoding='utf-8') as file:
        csv_reader = csv.DictReader(file)
        
        for row in csv_reader:
            try:
                # Procesar ubicación
                ubicacion_id = insert_ubicacion(conn, row['ubicacion'])
                
                # Procesar tipo de carrocería
                carroceria_id = insert_tipo_carroceria(conn, row['tipo_carroceria'])
                
                # Determinar tipo de motor
                tipo_motor = "Desconocido"
                if to_boolean(row['combustible_Eléctrico']):
                    tipo_motor = "Eléctrico"
                elif to_boolean(row['combustible_Híbrido Enchufable']):
                    tipo_motor = "Híbrido Enchufable"
                elif to_boolean(row['combustible_Híbrido']):
                    tipo_motor = "Híbrido"
                elif to_boolean(row['combustible_Gasolina']):
                    tipo_motor = "Gasolina"
                elif to_boolean(row['combustible_Diésel']):
                    tipo_motor = "Diésel"
                elif to_boolean(row['combustible_Gas']):
                    tipo_motor = "Gas"
                
                # Preparar datos del vehículo
                vehiculo_data = {
                    'marca': row['marca'],
                    'modelo': row['modelo'],
                    'precio_contado': round(float(row['precio_contado']), 2),
                    'precio_final': round(float(row['precio_final']), 2),

                    'financiacion_disponible': to_boolean(row['financiacion_disponible']),
                    'descuento': to_float(row['descuento']) or 0.0,
                    'kilometraje': to_float(row['kilometraje']),
                    'ubicacion_id': ubicacion_id,
                    'mes_matriculacion': to_int(row['mes_matriculacion2']),
                    'ano_matriculacion': to_int(row['año_matriculacion2']),
                    'garantia': to_int(row['garantia_limpia']),
                    'transmision_cod': row['transmision_cod'],
                    'tipo_motor': tipo_motor,
                    'tipo_carroceria_id': carroceria_id,
                    'num_asientos': to_int(row['asientos']),
                    'potencia': to_int(row['potencia']),
                    'num_puertas': to_int(row['puertas']),
                    'es_km0': to_boolean(row['es_km0']),
                    'es_demo': to_boolean(row['es_demo']),
                    'combustible_diesel': to_boolean(row['combustible_Diésel']),
                    'combustible_electrico': to_boolean(row['combustible_Eléctrico']),
                    'combustible_gas': to_boolean(row['combustible_Gas']),
                    'combustible_gasolina': to_boolean(row['combustible_Gasolina']),
                    'combustible_hibrido': to_boolean(row['combustible_Híbrido']),
                    'combustible_hibrido_enchufable': to_boolean(row['combustible_Híbrido Enchufable'])
                    }

                
                # Insertar vehículo
                insert_vehiculo = """
                INSERT INTO vehiculos (
                    marca, modelo, precio_contado, precio_final, financiacion_disponible, descuento,
                    kilometraje, ubicacion_id, mes_matriculacion, ano_matriculacion, garantia,
                    transmision_cod, tipo_motor, tipo_carroceria_id, 
                    num_asientos, potencia, num_puertas,
                    es_km0, es_demo, 
                    combustible_diesel, combustible_electrico, combustible_gas, 
                    combustible_gasolina, combustible_hibrido, combustible_hibrido_enchufable
                ) VALUES (
                    %(marca)s, %(modelo)s, %(precio_contado)s, %(precio_final)s, %(financiacion_disponible)s, %(descuento)s,
                    %(kilometraje)s, %(ubicacion_id)s, %(mes_matriculacion)s, %(ano_matriculacion)s, %(garantia)s,
                    %(transmision_cod)s, %(tipo_motor)s, %(tipo_carroceria_id)s,
                    %(num_asientos)s, %(potencia)s, %(num_puertas)s,
                    %(es_km0)s, %(es_demo)s,
                    %(combustible_diesel)s, %(combustible_electrico)s, %(combustible_gas)s,
                    %(combustible_gasolina)s, %(combustible_hibrido)s, %(combustible_hibrido_enchufable)s
                )
                """ 
                
                cursor.execute(insert_vehiculo, vehiculo_data)
                conn.commit()
                
            except Exception as e:
                print(f"Error al procesar fila: {e}")
                print(f"Fila problemática: {row}")
                conn.rollback()
    
    cursor.close()
    conn.close()
    print("Proceso de carga de datos completado.")

# Ejecutar el proceso
if __name__ == "__main__":
    csv_file = "C:\\Users\\abner\\proyecto\\ejemplo-proyecto\\DB\\limpieza_completa.csv"  # Asegúrate de que el archivo esté en el mismo directorio
    process_csv_file(csv_file)

Proceso de carga de datos completado.
